In [2]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_DEV=1

env: ANYWIDGET_DEV=1


In [3]:
import numpy as np
from sequilt import Sequilt
from sequilt.model import EventGraph, EventCanvas, SequletModel, LabelModel

In [16]:
from datasets import load_dataset
from sequilt.data import get_ids, get_tokenizer

ds = load_dataset("neuralbioinfo/bacterial_promoters")
tokenizer = get_tokenizer(type="dna", k=1)
ids, tokens = get_ids(
  ds["test_multispecies"]["segment"], tokenizer, max_tokens=32
)
labels = [
  LabelModel(value=value, name=name)
  for value, name in tokenizer._id_to_token.items()
  if value != 0
]

labels = sorted(labels, key=lambda x: x.name)

100%|██████████| 22582/22582 [00:01<00:00, 22129.43it/s]


In [26]:
# from datasets import load_dataset
# from sequilt.data import get_ids, get_tokenizer, get_featured_ids

# ds = load_dataset("ajaykarthick/imdb-movie-reviews")
# tokenizer = get_tokenizer(type="language")
# ids, tokens = get_ids(
#   ds['test']["review"], tokenizer, max_tokens=32
# )
# featured_ids = get_featured_ids(ids, tokenizer, "tf-idf", n_features=20)
# feature_mask = np.isin(ids, featured_ids)
# ids = np.where(feature_mask, ids, 0)
# labels = [
#   LabelModel(value=id, name=tokenizer.id_to_token(id))
#   for id in featured_ids
# ]

100%|██████████| 10000/10000 [00:05<00:00, 1942.60it/s]


In [41]:
G = EventGraph(ids)

canvas = EventCanvas(width=ids.shape[1], initial_height=ids.shape[0] * 2)

for event1, event2, cooccurence in G.sorted_edges:
  if cooccurence < 5:
    break
  sequlet = SequletModel([event1, event2])
  canvas.draw_sequlet(sequlet)
  G.remove_events_from([event1, event2])

In [37]:
print(len(G))

for node in G.nodes:
  sequlet = SequletModel([node])
  canvas.draw_sequlet(sequlet)

604


In [42]:
w = Sequilt(sequlets=canvas.sequlet_rects, labels=labels, width=800, height=400)
w

Sequilt(height=400, labels=[{'value': 59, 'name': 'movie'}, {'value': 5, 'name': 'film'}, {'value': 30, 'name'…